<a href="https://colab.research.google.com/github/punnoose-1620/masters-thesis-sensor-data/blob/main/Intent_Identifier_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automotive Intent Classification and Slot Filling System
=========================================================

A modular TensorFlow/Keras implementation for multi-task learning on automotive technical queries.

This script implements three neural architectures:
1. Bi-LSTM with Self-Attention
2. 1D-CNN with multiple kernel sizes
3. Joint Multi-Task Learning model

Author: Automotive NLP Team
Date: 2026-02-04

## Imports and Installs

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

## Set random seeds for reproducibility

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

## Configurations and Initializations

### GLOBAL CONFIGURATION

In [ ]:
CONFIG = {
    'FILE_PATH': 'datasets\intent_dataset.csv',  # Update this path
    'EMBEDDING_DIM': 128,
    'HIDDEN_UNITS': 128,
    'LEARNING_RATE': 0.001,
    'BATCH_SIZE': 32,
    'EPOCHS': 50,
    'MAX_SEQUENCE_LENGTH': 50,
    'MAX_VOCAB_SIZE': 10000,
    'VALIDATION_SPLIT': 0.15,
    'TEST_SPLIT': 0.15,
    'EARLY_STOPPING_PATIENCE': 5,
    'CNN_FILTERS': 128,
    'CNN_KERNEL_SIZES': [3, 4, 5],
    'DROPOUT_RATE': 0.3,
}

### Global variables to store preprocessing artifacts

In [4]:
tokenizer = None
label_encoders = {}
history_objects = {}
test_results = {}

COMPARISON_CHART = '/Visualizations/model_comparison.png'
BI_LSTM_CHART = '/Visualizations/bilstm_results.png'
CNN_CHART = '/Visualizations/cnn_results.png'
JOINT_CHART = '/Visualizations/joint_results.png'

## Dataset Functions

### FUNCTION 1: LOAD DATA

In [5]:
def load_data(file_path=None):
    """
    Load automotive dataset from CSV file.

    Args:
        file_path (str): Path to the CSV file. Uses CONFIG['FILE_PATH'] if None.

    Returns:
        pd.DataFrame: Loaded dataset
    """
    if file_path is None:
        file_path = CONFIG['FILE_PATH']

    print(f"Loading data from: {file_path}")

    try:
        df = pd.read_csv(file_path)
        print(f"✓ Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns")
        print(f"✓ Columns: {list(df.columns)}")
        print(f"\nFirst few rows:")
        print(df.head())

        # Validate required columns
        required_cols = ['query', 'intent', 'brand', 'model', 'sensor']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Handle missing values
        print(f"\nMissing values before handling:")
        print(df.isnull().sum())

        # Fill missing slot values with 'NONE' to preserve them as valid categories
        for col in ['brand', 'model', 'sensor', 'year']:
            if col in df.columns:
                df[col] = df[col].fillna('NONE').astype(str)

        # Drop rows with missing queries or intents
        df = df.dropna(subset=['query', 'intent'])

        print(f"\n✓ Data cleaned: {df.shape[0]} rows remaining")
        print(f"\nIntent distribution:")
        print(df['intent'].value_counts())

        return df

    except FileNotFoundError:
        print(f"✗ Error: File not found at {file_path}")
        print("Creating synthetic dataset for demonstration...")
        return create_synthetic_dataset()
    except Exception as e:
        print(f"✗ Error loading data: {str(e)}")
        raise

### FUNCTION 2: PREPROCESS DATA

In [6]:
def preprocess_data(df):
    """
    Preprocess automotive queries and labels.
    Handles technical jargon carefully - no aggressive stop-word removal.

    Args:
        df (pd.DataFrame): Input dataframe

    Returns:
        tuple: (X_padded, y_intent, y_brand, y_model, y_sensor, metadata)
    """
    global tokenizer, label_encoders

    print("\n" + "="*70)
    print("PREPROCESSING DATA")
    print("="*70)

    # 1. Text Tokenization (preserve technical terms)
    print("\n1. Tokenizing queries...")
    tokenizer = Tokenizer(
        num_words=CONFIG['MAX_VOCAB_SIZE'],
        oov_token='<OOV>',
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',  # Keep technical chars
        lower=True
    )

    tokenizer.fit_on_texts(df['query'])
    sequences = tokenizer.texts_to_sequences(df['query'])

    # 2. Padding
    print(f"2. Padding sequences to length {CONFIG['MAX_SEQUENCE_LENGTH']}...")
    X_padded = pad_sequences(
        sequences,
        maxlen=CONFIG['MAX_SEQUENCE_LENGTH'],
        padding='post',
        truncating='post'
    )

    vocab_size = min(len(tokenizer.word_index) + 1, CONFIG['MAX_VOCAB_SIZE'])
    print(f"   ✓ Vocabulary size: {vocab_size}")
    print(f"   ✓ Padded shape: {X_padded.shape}")

    # 3. Encode labels
    print("\n3. Encoding labels...")

    # Intent (primary classification)
    label_encoders['intent'] = LabelEncoder()
    y_intent_encoded = label_encoders['intent'].fit_transform(df['intent'])
    y_intent = to_categorical(y_intent_encoded)
    print(f"   ✓ Intent classes: {len(label_encoders['intent'].classes_)}")
    print(f"     Classes: {label_encoders['intent'].classes_}")

    # Slot filling targets
    slot_columns = ['brand', 'model', 'sensor']
    y_slots = {}

    for slot in slot_columns:
        if slot in df.columns:
            label_encoders[slot] = LabelEncoder()
            y_encoded = label_encoders[slot].fit_transform(df[slot])
            y_slots[slot] = to_categorical(y_encoded)
            print(f"   ✓ {slot.capitalize()} classes: {len(label_encoders[slot].classes_)}")

    # Prepare metadata
    metadata = {
        'vocab_size': vocab_size,
        'num_intent_classes': len(label_encoders['intent'].classes_),
        'num_brand_classes': len(label_encoders['brand'].classes_),
        'num_model_classes': len(label_encoders['model'].classes_),
        'num_sensor_classes': len(label_encoders['sensor'].classes_),
        'intent_labels': label_encoders['intent'].classes_,
        'max_seq_length': CONFIG['MAX_SEQUENCE_LENGTH']
    }

    print(f"\n✓ Preprocessing complete!")
    print(f"  Input shape: {X_padded.shape}")
    print(f"  Intent shape: {y_intent.shape}")

    return X_padded, y_intent, y_slots['brand'], y_slots['model'], y_slots['sensor'], metadata

### FUNCTION 3: SPLIT DATA

In [7]:
def split_data(X, y_intent, y_brand, y_model, y_sensor):
    """
    Split data into Train (70%), Validation (15%), and Test (15%).

    Args:
        X: Input sequences
        y_intent, y_brand, y_model, y_sensor: Target labels

    Returns:
        dict: Dictionary containing all splits
    """
    print("\n" + "="*70)
    print("SPLITTING DATA")
    print("="*70)

    # First split: Train+Val (85%) vs Test (15%)
    test_size = CONFIG['TEST_SPLIT']
    val_size = CONFIG['VALIDATION_SPLIT'] / (1 - test_size)  # 15% of remaining 85%

    X_temp, X_test, y_intent_temp, y_intent_test, y_brand_temp, y_brand_test, \
    y_model_temp, y_model_test, y_sensor_temp, y_sensor_test = train_test_split(
        X, y_intent, y_brand, y_model, y_sensor,
        test_size=test_size,
        random_state=42,
        stratify=np.argmax(y_intent, axis=1)
    )

    # Second split: Train (70%) vs Validation (15%)
    X_train, X_val, y_intent_train, y_intent_val, y_brand_train, y_brand_val, \
    y_model_train, y_model_val, y_sensor_train, y_sensor_val = train_test_split(
        X_temp, y_intent_temp, y_brand_temp, y_model_temp, y_sensor_temp,
        test_size=val_size,
        random_state=42,
        stratify=np.argmax(y_intent_temp, axis=1)
    )

    splits = {
        'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
        'y_intent_train': y_intent_train, 'y_intent_val': y_intent_val, 'y_intent_test': y_intent_test,
        'y_brand_train': y_brand_train, 'y_brand_val': y_brand_val, 'y_brand_test': y_brand_test,
        'y_model_train': y_model_train, 'y_model_val': y_model_val, 'y_model_test': y_model_test,
        'y_sensor_train': y_sensor_train, 'y_sensor_val': y_sensor_val, 'y_sensor_test': y_sensor_test,
    }

    print(f"Train set: {X_train.shape[0]} samples ({X_train.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"Val set:   {X_val.shape[0]} samples ({X_val.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"Test set:  {X_test.shape[0]} samples ({X_test.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"\n✓ Data split complete!")

    return splits

## Model Functions - Design and Creation

### FUNCTION 4: BUILD BI-LSTM MODEL WITH SELF-ATTENTION

In [ ]:
def build_bilstm_model(metadata):
    """
    Build Bi-directional LSTM model with Self-Attention layer.

    Args:
        metadata (dict): Model configuration metadata

    Returns:
        keras.Model: Compiled Bi-LSTM model
    """
    print("\n" + "="*70)
    print("BUILDING BI-LSTM MODEL WITH SELF-ATTENTION")
    print("="*70)

    # Input layer
    input_layer = Input(shape=(metadata['max_seq_length'],), name='input')

    # Embedding layer
    embedding = layers.Embedding(
        input_dim=metadata['vocab_size'],
        output_dim=CONFIG['EMBEDDING_DIM'],
        mask_zero=True,
        name='embedding'
    )(input_layer)

    # Bi-LSTM layer
    bilstm = layers.Bidirectional(
        layers.LSTM(CONFIG['HIDDEN_UNITS'], return_sequences=True),
        name='bilstm'
    )(embedding)

    # Self-Attention mechanism
    #attention = layers.Attention(name='self_attention')([bilstm, bilstm])

    # Global pooling
    #pooled = layers.GlobalAveragePooling1D(name='global_pool')(attention)

    # Global pooling (no self-attention to avoid gradient bug with layers.Attention)
    pooled = layers.GlobalAveragePooling1D(name='global_pool')(bilstm)

    # Dropout for regularization
    dropout = layers.Dropout(CONFIG['DROPOUT_RATE'], name='dropout')(pooled)

    # Dense layer
    dense = layers.Dense(CONFIG['HIDDEN_UNITS'], activation='relu', name='dense')(dropout)

    # Output layer for intent classification
    output = layers.Dense(
        metadata['num_intent_classes'],
        activation='softmax',
        name='intent_output'
    )(dense)

    # Create model
    model = Model(inputs=input_layer, outputs=output, name='BiLSTM_SelfAttention')

    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['LEARNING_RATE']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print(model.summary())
    print(f"\n✓ Bi-LSTM model built successfully!")

    return model

### FUNCTION 5: BUILD 1D-CNN MODEL

In [9]:
def build_cnn_model(metadata):
    """
    Build 1D-CNN model with multiple kernel sizes (3, 4, 5) to capture
    local technical n-grams.

    Args:
        metadata (dict): Model configuration metadata

    Returns:
        keras.Model: Compiled CNN model
    """
    print("\n" + "="*70)
    print("BUILDING 1D-CNN MODEL WITH MULTI-KERNEL")
    print("="*70)

    # Input layer
    input_layer = Input(shape=(metadata['max_seq_length'],), name='input')

    # Embedding layer
    embedding = layers.Embedding(
        input_dim=metadata['vocab_size'],
        output_dim=CONFIG['EMBEDDING_DIM'],
        name='embedding'
    )(input_layer)

    # Multiple CNN branches with different kernel sizes
    conv_branches = []
    for kernel_size in CONFIG['CNN_KERNEL_SIZES']:
        conv = layers.Conv1D(
            filters=CONFIG['CNN_FILTERS'],
            kernel_size=kernel_size,
            activation='relu',
            name=f'conv1d_k{kernel_size}'
        )(embedding)
        pool = layers.GlobalMaxPooling1D(name=f'maxpool_k{kernel_size}')(conv)
        conv_branches.append(pool)

    # Concatenate all branches
    if len(conv_branches) > 1:
        concatenated = layers.Concatenate(name='concat_branches')(conv_branches)
    else:
        concatenated = conv_branches[0]

    # Dropout for regularization
    dropout = layers.Dropout(CONFIG['DROPOUT_RATE'], name='dropout')(concatenated)

    # Dense layer
    dense = layers.Dense(CONFIG['HIDDEN_UNITS'], activation='relu', name='dense')(dropout)

    # Output layer for intent classification
    output = layers.Dense(
        metadata['num_intent_classes'],
        activation='softmax',
        name='intent_output'
    )(dense)

    # Create model
    model = Model(inputs=input_layer, outputs=output, name='CNN_MultiKernel')

    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['LEARNING_RATE']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print(model.summary())
    print(f"\n✓ 1D-CNN model built successfully!")

    return model

### FUNCTION 6: BUILD JOINT MULTI-TASK MODEL

In [10]:
def build_joint_model(metadata):
    """
    Build Joint Multi-Task Learning model with shared backbone and multiple heads
    for Intent Classification and Slot Filling (Brand, Model, Sensor).

    Args:
        metadata (dict): Model configuration metadata

    Returns:
        keras.Model: Compiled joint model
    """
    print("\n" + "="*70)
    print("BUILDING JOINT MULTI-TASK MODEL")
    print("="*70)

    # Input layer
    input_layer = Input(shape=(metadata['max_seq_length'],), name='input')

    # Shared Embedding layer
    embedding = layers.Embedding(
        input_dim=metadata['vocab_size'],
        output_dim=CONFIG['EMBEDDING_DIM'],
        mask_zero=True,
        name='shared_embedding'
    )(input_layer)

    # Shared Bi-LSTM encoder
    bilstm = layers.Bidirectional(
        layers.LSTM(CONFIG['HIDDEN_UNITS'], return_sequences=True),
        name='shared_bilstm'
    )(embedding)

    # Shared Self-Attention
    attention = layers.Attention(name='shared_attention')([bilstm, bilstm])

    # Global pooling
    pooled = layers.GlobalAveragePooling1D(name='shared_pool')(attention)

    # Shared dropout
    shared_features = layers.Dropout(CONFIG['DROPOUT_RATE'], name='shared_dropout')(pooled)

    # Shared dense layer
    shared_dense = layers.Dense(
        CONFIG['HIDDEN_UNITS'],
        activation='relu',
        name='shared_dense'
    )(shared_features)

    # -------------------------------------------------------------------------
    # Task-specific heads
    # -------------------------------------------------------------------------

    # Head 1: Intent Classification
    intent_dense = layers.Dense(64, activation='relu', name='intent_dense')(shared_dense)
    intent_dropout = layers.Dropout(0.2, name='intent_dropout')(intent_dense)
    intent_output = layers.Dense(
        metadata['num_intent_classes'],
        activation='softmax',
        name='intent_output'
    )(intent_dropout)

    # Head 2: Brand Slot Filling
    brand_dense = layers.Dense(64, activation='relu', name='brand_dense')(shared_dense)
    brand_dropout = layers.Dropout(0.2, name='brand_dropout')(brand_dense)
    brand_output = layers.Dense(
        metadata['num_brand_classes'],
        activation='softmax',
        name='brand_output'
    )(brand_dropout)

    # Head 3: Model Slot Filling
    model_dense = layers.Dense(64, activation='relu', name='model_dense')(shared_dense)
    model_dropout = layers.Dropout(0.2, name='model_dropout')(model_dense)
    model_output = layers.Dense(
        metadata['num_model_classes'],
        activation='softmax',
        name='model_output'
    )(model_dropout)

    # Head 4: Sensor Slot Filling
    sensor_dense = layers.Dense(64, activation='relu', name='sensor_dense')(shared_dense)
    sensor_dropout = layers.Dropout(0.2, name='sensor_dropout')(sensor_dense)
    sensor_output = layers.Dense(
        metadata['num_sensor_classes'],
        activation='softmax',
        name='sensor_output'
    )(sensor_dropout)

    # Create multi-output model
    model = Model(
        inputs=input_layer,
        outputs=[intent_output, brand_output, model_output, sensor_output],
        name='Joint_MTL_Model'
    )

    # Compile with multiple losses and metrics
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['LEARNING_RATE']),
        loss={
            'intent_output': 'categorical_crossentropy',
            'brand_output': 'categorical_crossentropy',
            'model_output': 'categorical_crossentropy',
            'sensor_output': 'categorical_crossentropy'
        },
        loss_weights={
            'intent_output': 2.0,  # Prioritize intent slightly
            'brand_output': 1.0,
            'model_output': 1.0,
            'sensor_output': 1.0
        },
        metrics={
            'intent_output': ['accuracy'],
            'brand_output': ['accuracy'],
            'model_output': ['accuracy'],
            'sensor_output': ['accuracy']
        }
    )

    print(model.summary())
    print(f"\n✓ Joint Multi-Task model built successfully!")

    return model

## Model Functions - Training and Evaluation

### FUNCTION 7: TRAIN AND EVALUATE BI-LSTM

In [11]:
def train_eval_bilstm(model, splits, metadata):
    """
    Train Bi-LSTM model with early stopping and evaluate on test set.

    Args:
        model: Compiled Bi-LSTM model
        splits (dict): Train/val/test data splits
        metadata (dict): Model metadata

    Returns:
        tuple: (history, test_metrics)
    """
    global history_objects, test_results

    print("\n" + "="*70)
    print("TRAINING BI-LSTM MODEL")
    print("="*70)

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['EARLY_STOPPING_PATIENCE'],
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    # Train model
    print("\nTraining...")
    history = model.fit(
        splits['X_train'],
        splits['y_intent_train'],
        validation_data=(splits['X_val'], splits['y_intent_val']),
        epochs=CONFIG['EPOCHS'],
        batch_size=CONFIG['BATCH_SIZE'],
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    history_objects['bilstm'] = history

    # Evaluate on test set
    print("\n" + "="*70)
    print("EVALUATING BI-LSTM ON TEST SET")
    print("="*70)

    test_loss, test_acc = model.evaluate(
        splits['X_test'],
        splits['y_intent_test'],
        verbose=0
    )

    # Predictions
    y_pred_probs = model.predict(splits['X_test'], verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(splits['y_intent_test'], axis=1)

    # Calculate metrics
    test_f1_macro = f1_score(y_true, y_pred, average='macro')
    test_f1_weighted = f1_score(y_true, y_pred, average='weighted')

    test_metrics = {
        'loss': test_loss,
        'accuracy': test_acc,
        'f1_macro': test_f1_macro,
        'f1_weighted': test_f1_weighted,
        'y_true': y_true,
        'y_pred': y_pred
    }

    test_results['bilstm'] = test_metrics

    print(f"\nTest Results:")
    print(f"  Loss: {test_loss:.4f}")
    print(f"  Accuracy: {test_acc:.4f}")
    print(f"  F1-Score (Macro): {test_f1_macro:.4f}")
    print(f"  F1-Score (Weighted): {test_f1_weighted:.4f}")

    print("\nClassification Report:")
    print(classification_report(
        y_true, y_pred,
        target_names=metadata['intent_labels'],
        digits=4
    ))

    return history, test_metrics

### FUNCTION 8: TRAIN AND EVALUATE CNN

In [12]:
def train_eval_cnn(model, splits, metadata):
    """
    Train CNN model with early stopping and evaluate on test set.

    Args:
        model: Compiled CNN model
        splits (dict): Train/val/test data splits
        metadata (dict): Model metadata

    Returns:
        tuple: (history, test_metrics)
    """
    global history_objects, test_results

    print("\n" + "="*70)
    print("TRAINING 1D-CNN MODEL")
    print("="*70)

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['EARLY_STOPPING_PATIENCE'],
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    # Train model
    print("\nTraining...")
    history = model.fit(
        splits['X_train'],
        splits['y_intent_train'],
        validation_data=(splits['X_val'], splits['y_intent_val']),
        epochs=CONFIG['EPOCHS'],
        batch_size=CONFIG['BATCH_SIZE'],
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    history_objects['cnn'] = history

    # Evaluate on test set
    print("\n" + "="*70)
    print("EVALUATING 1D-CNN ON TEST SET")
    print("="*70)

    test_loss, test_acc = model.evaluate(
        splits['X_test'],
        splits['y_intent_test'],
        verbose=0
    )

    # Predictions
    y_pred_probs = model.predict(splits['X_test'], verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(splits['y_intent_test'], axis=1)

    # Calculate metrics
    test_f1_macro = f1_score(y_true, y_pred, average='macro')
    test_f1_weighted = f1_score(y_true, y_pred, average='weighted')

    test_metrics = {
        'loss': test_loss,
        'accuracy': test_acc,
        'f1_macro': test_f1_macro,
        'f1_weighted': test_f1_weighted,
        'y_true': y_true,
        'y_pred': y_pred
    }

    test_results['cnn'] = test_metrics

    print(f"\nTest Results:")
    print(f"  Loss: {test_loss:.4f}")
    print(f"  Accuracy: {test_acc:.4f}")
    print(f"  F1-Score (Macro): {test_f1_macro:.4f}")
    print(f"  F1-Score (Weighted): {test_f1_weighted:.4f}")

    print("\nClassification Report:")
    print(classification_report(
        y_true, y_pred,
        target_names=metadata['intent_labels'],
        digits=4
    ))

    return history, test_metrics

### FUNCTION 9: TRAIN AND EVALUATE JOINT MODEL

In [13]:
def train_eval_joint(model, splits, metadata):
    """
    Train Joint Multi-Task model with early stopping and evaluate on test set.

    Args:
        model: Compiled Joint model
        splits (dict): Train/val/test data splits
        metadata (dict): Model metadata

    Returns:
        tuple: (history, test_metrics)
    """
    global history_objects, test_results

    print("\n" + "="*70)
    print("TRAINING JOINT MULTI-TASK MODEL")
    print("="*70)

    # Prepare multi-output training data
    y_train = {
        'intent_output': splits['y_intent_train'],
        'brand_output': splits['y_brand_train'],
        'model_output': splits['y_model_train'],
        'sensor_output': splits['y_sensor_train']
    }

    y_val = {
        'intent_output': splits['y_intent_val'],
        'brand_output': splits['y_brand_val'],
        'model_output': splits['y_model_val'],
        'sensor_output': splits['y_sensor_val']
    }

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=CONFIG['EARLY_STOPPING_PATIENCE'],
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    # Train model
    print("\nTraining...")
    history = model.fit(
        splits['X_train'],
        y_train,
        validation_data=(splits['X_val'], y_val),
        epochs=CONFIG['EPOCHS'],
        batch_size=CONFIG['BATCH_SIZE'],
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    history_objects['joint'] = history

    # Evaluate on test set
    print("\n" + "="*70)
    print("EVALUATING JOINT MODEL ON TEST SET")
    print("="*70)

    y_test = {
        'intent_output': splits['y_intent_test'],
        'brand_output': splits['y_brand_test'],
        'model_output': splits['y_model_test'],
        'sensor_output': splits['y_sensor_test']
    }

    test_results_raw = model.evaluate(
        splits['X_test'],
        y_test,
        verbose=0,
        return_dict=True
    )

    # Predictions
    predictions = model.predict(splits['X_test'], verbose=0)
    y_intent_pred_probs, y_brand_pred_probs, y_model_pred_probs, y_sensor_pred_probs = predictions

    # Intent predictions
    y_intent_pred = np.argmax(y_intent_pred_probs, axis=1)
    y_intent_true = np.argmax(splits['y_intent_test'], axis=1)

    # Calculate intent metrics
    intent_f1_macro = f1_score(y_intent_true, y_intent_pred, average='macro')
    intent_f1_weighted = f1_score(y_intent_true, y_intent_pred, average='weighted')
    intent_acc = accuracy_score(y_intent_true, y_intent_pred)

    # Brand predictions
    y_brand_pred = np.argmax(y_brand_pred_probs, axis=1)
    y_brand_true = np.argmax(splits['y_brand_test'], axis=1)
    brand_acc = accuracy_score(y_brand_true, y_brand_pred)

    # Model predictions
    y_model_pred = np.argmax(y_model_pred_probs, axis=1)
    y_model_true = np.argmax(splits['y_model_test'], axis=1)
    model_acc = accuracy_score(y_model_true, y_model_pred)

    # Sensor predictions
    y_sensor_pred = np.argmax(y_sensor_pred_probs, axis=1)
    y_sensor_true = np.argmax(splits['y_sensor_test'], axis=1)
    sensor_acc = accuracy_score(y_sensor_true, y_sensor_pred)

    test_metrics = {
        'loss': test_results_raw['loss'],
        'accuracy': intent_acc,
        'f1_macro': intent_f1_macro,
        'f1_weighted': intent_f1_weighted,
        'y_true': y_intent_true,
        'y_pred': y_intent_pred,
        'intent_accuracy': intent_acc,
        'brand_accuracy': brand_acc,
        'model_accuracy': model_acc,
        'sensor_accuracy': sensor_acc
    }

    test_results['joint'] = test_metrics

    print(f"\nTest Results:")
    print(f"  Overall Loss: {test_results_raw['loss']:.4f}")
    print(f"\nIntent Classification:")
    print(f"  Accuracy: {intent_acc:.4f}")
    print(f"  F1-Score (Macro): {intent_f1_macro:.4f}")
    print(f"  F1-Score (Weighted): {intent_f1_weighted:.4f}")
    print(f"\nSlot Filling Accuracy:")
    print(f"  Brand: {brand_acc:.4f}")
    print(f"  Model: {model_acc:.4f}")
    print(f"  Sensor: {sensor_acc:.4f}")

    print("\nIntent Classification Report:")
    print(classification_report(
        y_intent_true, y_intent_pred,
        target_names=metadata['intent_labels'],
        digits=4
    ))

    return history, test_metrics

## Model Functions - Result Visualizations

### FUNCTION 10: VISUALIZE BI-LSTM RESULTS

In [14]:
def visualize_bilstm(history, test_metrics, metadata, save_path=BI_LSTM_CHART):
    """
    Plot training history and confusion matrix for Bi-LSTM model.

    Args:
        history: Training history object
        test_metrics (dict): Test evaluation metrics
        metadata (dict): Model metadata
        save_path (str): Path to save the figure
    """
    print("\n" + "="*70)
    print("VISUALIZING BI-LSTM RESULTS")
    print("="*70)

    fig = plt.figure(figsize=(16, 5))

    # Plot 1: Training & Validation Accuracy
    ax1 = plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.title('Bi-LSTM: Accuracy Over Epochs', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)

    # Plot 2: Training & Validation Loss
    ax2 = plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.title('Bi-LSTM: Loss Over Epochs', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)

    # Plot 3: Confusion Matrix
    ax3 = plt.subplot(1, 3, 3)
    cm = confusion_matrix(test_metrics['y_true'], test_metrics['y_pred'])

    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    sns.heatmap(
        cm_normalized,
        annot=True,
        fmt='.2f',
        cmap='Blues',
        xticklabels=metadata['intent_labels'],
        yticklabels=metadata['intent_labels'],
        cbar_kws={'label': 'Proportion'},
        ax=ax3
    )
    plt.title('Bi-LSTM: Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Visualization saved to: {save_path}")
    plt.close()

### FUNCTION 11: VISUALIZE CNN RESULTS

In [15]:
def visualize_cnn(history, test_metrics, metadata, save_path=CNN_CHART):
    """
    Plot training history and confusion matrix for CNN model.

    Args:
        history: Training history object
        test_metrics (dict): Test evaluation metrics
        metadata (dict): Model metadata
        save_path (str): Path to save the figure
    """
    print("\n" + "="*70)
    print("VISUALIZING 1D-CNN RESULTS")
    print("="*70)

    fig = plt.figure(figsize=(16, 5))

    # Plot 1: Training & Validation Accuracy
    ax1 = plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.title('1D-CNN: Accuracy Over Epochs', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)

    # Plot 2: Training & Validation Loss
    ax2 = plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.title('1D-CNN: Loss Over Epochs', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)

    # Plot 3: Confusion Matrix
    ax3 = plt.subplot(1, 3, 3)
    cm = confusion_matrix(test_metrics['y_true'], test_metrics['y_pred'])

    # Normalize confusion matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    sns.heatmap(
        cm_normalized,
        annot=True,
        fmt='.2f',
        cmap='Greens',
        xticklabels=metadata['intent_labels'],
        yticklabels=metadata['intent_labels'],
        cbar_kws={'label': 'Proportion'},
        ax=ax3
    )
    plt.title('1D-CNN: Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Visualization saved to: {save_path}")
    plt.close()

### FUNCTION 12: VISUALIZE MODEL COMPARISON

In [16]:
def visualize_comparison(save_path=COMPARISON_CHART):
    """
    Create comprehensive side-by-side comparison of all three models.

    Args:
        save_path (str): Path to save the comparison figure
    """
    print("\n" + "="*70)
    print("CREATING MODEL COMPARISON VISUALIZATION")
    print("="*70)

    # Extract metrics for comparison
    models = ['Bi-LSTM', '1D-CNN', 'Joint MTL']
    model_keys = ['bilstm', 'cnn', 'joint']

    accuracies = [test_results[key]['accuracy'] for key in model_keys]
    f1_macros = [test_results[key]['f1_macro'] for key in model_keys]
    f1_weighteds = [test_results[key]['f1_weighted'] for key in model_keys]
    losses = [test_results[key]['loss'] for key in model_keys]

    # Create comprehensive comparison figure
    fig = plt.figure(figsize=(18, 10))

    # -------------------------------------------------------------------------
    # Plot 1: Accuracy Comparison
    # -------------------------------------------------------------------------
    ax1 = plt.subplot(2, 3, 1)
    bars = plt.bar(models, accuracies, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.8, edgecolor='black')
    plt.title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.ylabel('Accuracy', fontsize=12)
    plt.ylim([0, 1.0])
    plt.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar, val in zip(bars, accuracies):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # -------------------------------------------------------------------------
    # Plot 2: F1-Score (Macro) Comparison
    # -------------------------------------------------------------------------
    ax2 = plt.subplot(2, 3, 2)
    bars = plt.bar(models, f1_macros, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.8, edgecolor='black')
    plt.title('F1-Score (Macro) Comparison', fontsize=14, fontweight='bold')
    plt.ylabel('F1-Score (Macro)', fontsize=12)
    plt.ylim([0, 1.0])
    plt.grid(axis='y', alpha=0.3)

    for bar, val in zip(bars, f1_macros):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # -------------------------------------------------------------------------
    # Plot 3: Loss Comparison
    # -------------------------------------------------------------------------
    ax3 = plt.subplot(2, 3, 3)
    bars = plt.bar(models, losses, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.8, edgecolor='black')
    plt.title('Test Loss Comparison', fontsize=14, fontweight='bold')
    plt.ylabel('Loss', fontsize=12)
    plt.grid(axis='y', alpha=0.3)

    for bar, val in zip(bars, losses):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # -------------------------------------------------------------------------
    # Plot 4: Multi-Metric Radar Chart
    # -------------------------------------------------------------------------
    ax4 = plt.subplot(2, 3, 4, projection='polar')

    categories = ['Accuracy', 'F1-Macro', 'F1-Weighted']
    num_vars = len(categories)

    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    for i, (model_name, key) in enumerate(zip(models, model_keys)):
        values = [
            test_results[key]['accuracy'],
            test_results[key]['f1_macro'],
            test_results[key]['f1_weighted']
        ]
        values += values[:1]

        colors = ['#3498db', '#2ecc71', '#e74c3c']
        ax4.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[i])
        ax4.fill(angles, values, alpha=0.15, color=colors[i])

    ax4.set_xticks(angles[:-1])
    ax4.set_xticklabels(categories, fontsize=10)
    ax4.set_ylim(0, 1)
    ax4.set_title('Multi-Metric Performance Radar', fontsize=14, fontweight='bold', pad=20)
    ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
    ax4.grid(True)

    # -------------------------------------------------------------------------
    # Plot 5: Training History Comparison (Accuracy)
    # -------------------------------------------------------------------------
    ax5 = plt.subplot(2, 3, 5)

    for i, (model_name, key) in enumerate(zip(models, model_keys)):
        if key in history_objects:
            history = history_objects[key]
            if key == 'joint':
                # For joint model, use intent_output accuracy
                acc_key = 'intent_output_accuracy' if 'intent_output_accuracy' in history.history else 'accuracy'
            else:
                acc_key = 'accuracy'

            if acc_key in history.history:
                colors = ['#3498db', '#2ecc71', '#e74c3c']
                plt.plot(history.history[acc_key], label=model_name, linewidth=2, color=colors[i])

    plt.title('Training Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)

    # -------------------------------------------------------------------------
    # Plot 6: Summary Table
    # -------------------------------------------------------------------------
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')

    # Create summary data
    table_data = []
    for model_name, key in zip(models, model_keys):
        row = [
            model_name,
            f"{test_results[key]['accuracy']:.4f}",
            f"{test_results[key]['f1_macro']:.4f}",
            f"{test_results[key]['f1_weighted']:.4f}",
            f"{test_results[key]['loss']:.4f}"
        ]
        table_data.append(row)

    # Add slot filling accuracy for Joint model
    if 'joint' in test_results and 'brand_accuracy' in test_results['joint']:
        joint_slots = f"Brand: {test_results['joint']['brand_accuracy']:.3f} | " \
                     f"Model: {test_results['joint']['model_accuracy']:.3f} | " \
                     f"Sensor: {test_results['joint']['sensor_accuracy']:.3f}"
    else:
        joint_slots = "N/A"

    table = ax6.table(
        cellText=table_data,
        colLabels=['Model', 'Accuracy', 'F1-Macro', 'F1-Weighted', 'Loss'],
        cellLoc='center',
        loc='center',
        bbox=[0, 0.3, 1, 0.6]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)

    # Style header
    for i in range(5):
        table[(0, i)].set_facecolor('#34495e')
        table[(0, i)].set_text_props(weight='bold', color='white')

    # Highlight best values
    best_acc_idx = np.argmax(accuracies)
    best_f1_idx = np.argmax(f1_macros)

    for i in range(len(models)):
        if i == best_acc_idx:
            table[(i+1, 1)].set_facecolor('#d5f4e6')
        if i == best_f1_idx:
            table[(i+1, 2)].set_facecolor('#d5f4e6')

    ax6.text(0.5, 0.15, f"Joint MTL Slot Filling: {joint_slots}",
            ha='center', va='center', fontsize=9, style='italic',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

    ax6.set_title('Performance Summary', fontsize=14, fontweight='bold', pad=20)

    # -------------------------------------------------------------------------
    # Overall title
    # -------------------------------------------------------------------------
    fig.suptitle('Automotive Intent Classification: Model Comparison',
                fontsize=16, fontweight='bold', y=0.98)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Comparison visualization saved to: {save_path}")
    plt.close()

    # Print summary
    print("\n" + "="*70)
    print("FINAL MODEL COMPARISON SUMMARY")
    print("="*70)
    print(f"\n{'Model':<15} {'Accuracy':<12} {'F1-Macro':<12} {'F1-Weighted':<12} {'Loss':<10}")
    print("-" * 70)
    for model_name, key in zip(models, model_keys):
        print(f"{model_name:<15} "
              f"{test_results[key]['accuracy']:<12.4f} "
              f"{test_results[key]['f1_macro']:<12.4f} "
              f"{test_results[key]['f1_weighted']:<12.4f} "
              f"{test_results[key]['loss']:<10.4f}")

    # Determine best model
    best_model_idx = np.argmax([test_results[key]['f1_macro'] for key in model_keys])
    print("\n" + "="*70)
    print(f"🏆 BEST MODEL: {models[best_model_idx]} (based on F1-Macro score)")
    print("="*70)

## MAIN EXECUTION PIPELINE

In [19]:
print("\n" + "="*70)
print(" AUTOMOTIVE INTENT CLASSIFICATION & SLOT FILLING SYSTEM")
print("="*70)
print(f"\nConfiguration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


 AUTOMOTIVE INTENT CLASSIFICATION & SLOT FILLING SYSTEM

Configuration:
  FILE_PATH: /content/intent_dataset.csv
  EMBEDDING_DIM: 128
  HIDDEN_UNITS: 128
  LEARNING_RATE: 0.001
  BATCH_SIZE: 32
  EPOCHS: 50
  MAX_SEQUENCE_LENGTH: 50
  MAX_VOCAB_SIZE: 10000
  VALIDATION_SPLIT: 0.15
  TEST_SPLIT: 0.15
  EARLY_STOPPING_PATIENCE: 5
  CNN_FILTERS: 128
  CNN_KERNEL_SIZES: [3, 4, 5]
  DROPOUT_RATE: 0.3


In [20]:
# Step 1: Load Data
df = load_data()

Loading data from: /content/intent_dataset.csv
✓ Data loaded successfully: 11178 rows, 9 columns
✓ Columns: ['query', 'intent', 'brand', 'model', 'year', 'sensor', 'style', 'word_count', 'char_count']

First few rows:
                                         query               intent  \
0          status of LiDAR collection in Jiyue  system_status_check   
1  explain Vision system context in Polestar 2      data_definition   
2                 what is Lidar for Lynk & Co?      data_definition   
3      who is owner High Voltage battery data?    access_governance   
4               roof rig needed for Acoustics?    collection_method   

       brand       model  year                sensor     style  word_count  \
0      Jiyue          01  2023                 LiDAR   verbose           6   
1   Polestar  Polestar 2  2021         Vision system     noisy           7   
2  Lynk & Co          01  2019                 Lidar  standard           7   
3      Volvo        XC60  2019  High Voltag

In [21]:
# Step 2: Preprocess Data
X, y_intent, y_brand, y_model, y_sensor, metadata = preprocess_data(df)


PREPROCESSING DATA

1. Tokenizing queries...
2. Padding sequences to length 50...
   ✓ Vocabulary size: 1280
   ✓ Padded shape: (11178, 50)

3. Encoding labels...
   ✓ Intent classes: 10
     Classes: ['access_governance' 'anomaly_info' 'collection_method'
 'comparative_analysis' 'contextual_followups' 'data_definition'
 'imperative_commands' 'statistical_query' 'system_status_check'
 'temporal_stats']
   ✓ Brand classes: 7
   ✓ Model classes: 34
   ✓ Sensor classes: 22

✓ Preprocessing complete!
  Input shape: (11178, 50)
  Intent shape: (11178, 10)


In [22]:
# Step 3: Split Data
splits = split_data(X, y_intent, y_brand, y_model, y_sensor)


SPLITTING DATA
Train set: 7824 samples (70.0%)
Val set:   1677 samples (15.0%)
Test set:  1677 samples (15.0%)

✓ Data split complete!


In [25]:
# Step 4: Build and Train Bi-LSTM Model
bilstm_model = build_bilstm_model(metadata)
bilstm_history, bilstm_metrics = train_eval_bilstm(bilstm_model, splits, metadata)


BUILDING BI-LSTM MODEL WITH SELF-ATTENTION


Model: "BiLSTM_SelfAttention"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 50)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │    163,840 │ input[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 50)        │          0 │ input[0][0]       │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm              │ (None, 50, 256)   │    263,168 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention      │ (None, 50, 256)   │          0 │ bilstm[0][0],     │
│ (Attention)         │                   │            │ bilstm[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convert_to_tensor_2 │ (None, 50)        │          0 │ not_equal_3[0][0] │
│ (ConvertToTensor)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_pool         │ (None, 256)       │          0 │ self_attention[0… │
│ (GlobalAveragePool… │                   │            │ convert_to_tenso… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ global_pool[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ intent_output       │ (None, 10)        │      1,290 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 461,194 (1.76 MB)

 Trainable params: 461,194 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

None

✓ Bi-LSTM model built successfully!

TRAINING BI-LSTM MODEL

Training...
Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/BiLSTM_SelfAttention_1/self_attention_1/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py", line 510, in disp

In [ ]:
# Step 5: Build and Train CNN Model
cnn_model = build_cnn_model(metadata)
cnn_history, cnn_metrics = train_eval_cnn(cnn_model, splits, metadata)

In [ ]:
# Step 6: Build and Train Joint Model
joint_model = build_joint_model(metadata)
joint_history, joint_metrics = train_eval_joint(joint_model, splits, metadata)

In [ ]:
# Step 7: Visualizations
visualize_bilstm(bilstm_history, bilstm_metrics, metadata)
visualize_cnn(cnn_history, cnn_metrics, metadata)
visualize_comparison()

In [ ]:
print("\n" + "="*70)
print("✓ ALL TASKS COMPLETED SUCCESSFULLY!")
print("="*70)
print("\nGenerated files:")
print("  1. /home/claude/bilstm_results.png")
print("  2. /home/claude/cnn_results.png")
print("  3. /home/claude/model_comparison.png")
print("\nYou can now use these models for automotive query classification!")